<a href="https://colab.research.google.com/github/Suhaila-Hassan/ITI-NLP-Lab/blob/main/NER_Model_Building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == 

In [2]:
!python -m spacy init config config.cfg --pipeline ner --lang en

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [3]:
import spacy
from spacy.tokens import DocBin
from spacy.training import Example
from datasets import load_dataset
from tqdm import tqdm

In [4]:
dataset = load_dataset("eriktks/conll2003")

train_dataset = dataset['train']
val_dataset = dataset['validation']
test_dataset = dataset['test']

print("Train size:", len(train_dataset))
print("Validation size:", len(val_dataset))
print("Test size:", len(test_dataset))

print(train_dataset[0])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

The repository for eriktks/conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/eriktks/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

Train size: 14041
Validation size: 3250
Test size: 3453
{'id': '0', 'tokens': ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7], 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0], 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}


In [5]:
nlp = spacy.blank("en")

def convert_to_spacy(dataset, output_path):
    doc_bin = DocBin()
    for example in tqdm(dataset):
        tokens = example["tokens"]
        tags = example["ner_tags"]
        doc = nlp.make_doc(" ".join(tokens))
        ents = []
        start = 0
        for token, label in zip(doc, tags):
            word = token.text
            end = start + len(word)
            if label != 0:
                label_str = dataset.features["ner_tags"].feature.names[label]
                if label_str.startswith("B-"):
                    ent_start = token.idx
                    ent_end = token.idx + len(token)
                    ent_label = label_str[2:]
                    ents.append(spacy.tokens.Span(doc, token.i, token.i + 1, label=ent_label))
            start += len(word) + 1
        doc.ents = ents
        doc_bin.add(doc)
    doc_bin.to_disk(output_path)

convert_to_spacy(train_dataset, "train.spacy")
convert_to_spacy(val_dataset, "dev.spacy")
convert_to_spacy(test_dataset, "test.spacy")

100%|██████████| 3453/3453 [00:02<00:00, 1278.17it/s]


In [7]:
!python -m spacy train config.cfg --output ./output --gpu-id 0

ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     44.28    0.00    0.00    0.00    0.00
  0     200         64.39   3269.00   47.80   52.98   43.54    0.48
  0     400        152.13   2470.51   62.92   64.84   61.11    0.63
  0     600        207.17   2467.63   69.43   71.07   67.87    0.69
  0     800        254.89   2628.33   70.80   71.55   70.06    0.71
  0    1000        304.07   3086.20   73.10   75.99   70.41    0.73
  1    1200        370.28   3296.97   76.18   77.39   75.01    0.76
  1    1400        458.24   3133.18   75.96   78.20   73.85    0.76
  1    1600        521.99   

In [9]:
!python -m spacy evaluate output/model-best test.spacy --gpu-id 0

ℹ Using GPU: 0

================================== Results ==================================

TOK     100.00
NER P   74.94 
NER R   71.28 
NER F   73.07 
SPEED   21299 


=============================== NER (per type) ===============================

           P       R       F
LOC    77.58   76.14   76.85
ORG    77.41   67.25   71.97
PER    72.92   72.60   72.76
MISC   68.18   66.24   67.20

